In [69]:
import faro
import faro.proto.proto_types as pt
import os
import pyvision as pv
import cv2
import time
import csv

In [70]:
class FaroDetectionOptions(object):
    #user could change these variable : User-defined variables
    def __init__(self):
        #root file
        self.results_root_path = '/home/nv5/Research/FaceRecognition/faro/Notebooks/results_dlib'
        self.csvfiles_log = 'csvfiles_log'
        #Directory to save images with bounding boxes
        self.detect_log = 'detect_log'
        #Directory to save cropped face images (128*128)
        self.face_log = 'face_log'
        #Set to True if you want to save only the detection with 
        #the highest detection score (confidence)
        self.best = False
        #Detection Thresh - Retinaface uses 0.5 in their examples
        self.detect_thresh = 0.5
        #Set min size. If the bouding box of the face detected is
        #less that min_size then it will be discarded.
        self.min_size = 0
        #Set the maximum number of images it can process
        self.max_images = None
        self.max_size = None

class FaroClientConnectionOptions(object):
    
    def __init__(self):
        self.max_async = faro.DEFAULT_MAX_ASYNC
        self.max_message_size = faro.DEFAULT_MAX_MESSAGE_SIZE
        self.detect_port = faro.DEFAULT_PORT
        self.rec_port = faro.DEFAULT_PORT
        self.verbose = True

        

In [71]:
client_options = FaroClientConnectionOptions()
face_client = faro.FaceClient(client_options)
is_ready,status = face_client.status(verbose=client_options.verbose)
if not is_ready:
    print("ERROR: the FaRO service is not ready.")
    print(status)
    exit(-1)
else:
    if client_options.verbose:
        print('Connection to FaRO service established. [ algorithm: %s ]'%(status.algorithm))



<bound method FaceClient.status of <faro.FaceClient.FaceClient object at 0x7f7d11ce97f0>>
<class 'faro.proto.face_service_pb2.FaceServiceInfo'> status: READY
detection_support: true
extract_support: true
score_support: true
score_type: L2
match_threshold: 0.6000000238418579
algorithm: "DLIB_19.18.0"

Connection to FaRO service established. [ algorithm: DLIB_19.18.0 ]


In [72]:
#check if the input file is a image or video
def getFilesToProcess(args):
    images = []
    videos = []
    for each in args:
        print(each)
        if os.path.isfile(each) and pv.isImage(each):
            images.append(each)
        elif os.path.isfile(each) and pv.isVideo(each):
            videos.append(each)
        elif os.path.isdir(each):
            for path,dirs,files in os.walk(each):
                for filename in files:
                    #print(filename)
                    filepath = os.path.join(path,filename)
                    if os.path.isfile(filepath) and pv.isImage(filepath):
                        images.append(filepath)
                    if os.path.isfile(filepath) and pv.isVideo(filepath):
                        videos.append(filepath)
        else :
            raise ValueError("Cannot determine filetype:"+each)

    print("Found %d images and %d videos."%(len(images),len(videos)))
    return images, videos

In [73]:
def preprocessImage(im_interest, doptions):
    """
    If the doptions.max_size is defined then 
    if the image width or height is greater then
    doptions.max then it will be resized with 
    w and h equal to doptions.max
    """
    scale = 1.0
    while max(*im_interest.shape[:2]) > doptions.max_size:
        if max(*im_interest.shape[:2]) > 2*doptions.max_size:
            im = cv2.pyrDown(im)
            scale *= 0.5
        else:
            w,h = im.shape[:2]
            s = doptions.max_size/max(w,h)
            scale *= s
            w = int(s*w)
            h = int(s*h)
            im = cv2.resize(im,(w,h))
    return im

In [74]:
detection_options = FaroDetectionOptions()
if not os.path.isdir(detection_options.results_root_path):
    os.makedirs(detection_options.results_root_path)
#define data path 
data_dir = ['/home/nv5/Research/FaceRecognition/faro/bin/data/']
#get input file to process
image_list, video_list = getFilesToProcess(data_dir)

/home/nv5/Research/FaceRecognition/faro/bin/data/
Found 10 images and 1 videos.


In [75]:
def process_image_detections(each):
    """
    Detections returned by the detector are written into a csvfile
    Images annotated with bouding box are saved to a directory if options.detect_log is True
    Face region is extracted and resized to 128*128*3 and save to a directory of options.face_log is True
    """
    im, results, options, media_type = each
    if results.done():
        recs = results.result().face_records
        i = 0
        dimg = None
        csv_file = None
        for idx, face in enumerate(recs):
            base_name, ext = os.path.splitext(os.path.basename(face.source))
            # Filter faces based on min size
            size = min(face.detection.location.width,face.detection.location.height)
            if size < options.min_size:
                continue
    
            # Process Detections
            if csv_file is None:
                image_detection_file = open(os.path.join(options.results_root_path, 
                                                         options.csvfiles_log, media_type, base_name + '.csv'), 'w')
                file_identifier = csv.writer(image_detection_file)
                
                csv_header = ['source','frame','detect_id','type','score','x','y','w','h']
                if len(face.landmarks) > 0:
                    
                    for each_lpt in face.landmarks:
                        pt_id_label = each_lpt.landmark_id
                        xpt_label = pt_id_label + '_x'
                        ypt_label = pt_id_label + '_y'
                        csv_header.append(pt_id_label)
                        csv_header.append(xpt_label)
                        csv_header.append(ypt_label)
                    
                file_identifier.writerow(csv_header)
                csv_file = -1
                        
                                 
            csv_eachline = [face.source,
                            face.frame,
                            i,
                            face.detection.detection_class,
                            face.detection.score,
                            face.detection.location.x,
                            face.detection.location.y,
                            face.detection.location.width,
                            face.detection.location.height]
            
            if len(face.landmarks) > 0:
                for each_lpt in face.landmarks:
                    pt_id_label = each_lpt.landmark_id
                    xpt_label = each_lpt.location.x
                    ypt_label = each_lpt.location.y
                    csv_eachline.append(pt_id_label)
                    csv_eachline.append(xpt_label)
                    csv_eachline.append(ypt_label)
                    
            file_identifier.writerow(csv_eachline)
            
                
            if options.detect_log:
                detect_log_dir = os.path.join(options.results_root_path, options.detect_log, media_type)
                if not os.path.exists(detect_log_dir):
                    os.makedirs(detect_log_dir, exist_ok=True)
                rect = pt.rect_proto2pv(face.detection.location)
                if dimg is None:
                    dimg = pv.Image(im[:,:,::-1])
                dimg.annotateThickRect(rect)
                dimg.annotateLabel(pv.Point(rect.x+5,rect.y+5),face.detection.detection_class)
                dimg.annotateLabel(pv.Point(rect.x+5,rect.y+20),
                                    "Score: %0.4f"%(face.detection.score,), color='yellow')
                if len(face.landmarks) > 0:
                    for each_lmark in face.landmarks:
                            dimg.annotateCircle(pv.Point(each_lmark.location.x, each_lmark.location.y), 
                                                radius=3, color = 'green', fill='green')
                
            
            if options.face_log:
                face_log_dir = os.path.join(options.results_root_path, options.face_log, media_type)
                if not os.path.exists(face_log_dir):
                    os.makedirs(face_log_dir, exist_ok=True)
                #print(face.detection.location)
                
                rect = pt.rect_proto2pv(face.detection.location)
                rect = rect.rescale(1.5)
                affine = pv.AffineFromRect(rect,(128,128))
    
                try:
                    pvim = pv.Image(im[:,:,::-1])
                    view = affine(pvim)
                    if len(face.landmarks) > 0:
                        for each_lmark in face.landmarks:
                            transformed_lmarks = affine(pv.Point(each_lmark.location.x, each_lmark.location.y))
                            view.annotateCircle(transformed_lmarks, radius=3, color = 'green', fill='green')
  
                    base_name, ext = os.path.splitext(os.path.basename(face.source))
                    out_path = os.path.join(face_log_dir,
                                            os.path.basename(base_name)+'_face_%03d'%(face.detection.detection_id,)+ext)
                    
                    view.asAnnotated().save(out_path)
                    print('Saving face:',out_path)
                except:
                    print("WARNING: Image not processed correctly:",face.source)
            i += 1
        
        if options.detect_log:
            dimg.asAnnotated().save(os.path.join(detect_log_dir,
                                                     os.path.basename(base_name) + ext))
        image_detection_file.close()
        return False
    return True


In [76]:
def process_images(ilist, doptions):
    """
    Looping over images in the dataset and making client calls to the service
    """
    image_count = 0
    detect_queue = []
    start_time = time.time()
    for each_img in ilist:
        im = cv2.imread(each_img)
        
        if im is None:
            continue
        # convert BGR to RGB
        im = im[:,:,::-1]
        if doptions.max_size is not None:
            im = preprocessImage(im, doptions)
        results = face_client.detect(im, best=doptions.best, 
                             threshold=doptions.detect_thresh, 
                             min_size=doptions.min_size,
                             run_async=True, 
                             source=each_img, 
                             frame=-1)
        detect_queue.append([im, results, doptions, 'image'])
        detect_queue = list(filter(process_image_detections, detect_queue))
        image_count += 1
        if doptions.max_images is not None and image_count >= options.max_images:
            break 
            
    while len(detect_queue):
        detect_queue = list(filter(process_image_detections,detect_queue))
        time.sleep(0.05)
        
    end_time = time.time()
    print("Processed %d images in %0.3f seconds: %f images/second"%(image_count,end_time - start_time,
                                                                    image_count/(end_time - start_time)))
        

In [77]:
if not os.path.isdir(os.path.join(detection_options.results_root_path, detection_options.csvfiles_log, 'image')):
    os.makedirs(os.path.join(detection_options.results_root_path, detection_options.csvfiles_log, 'image'))
process_images(image_list, detection_options)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: local variable 'detection_threshold' referenced before assignment"
	debug_error_string = "{"created":"@1592914460.757749421","description":"Error received from peer ipv4:0.0.0.0:50030","file":"src/core/lib/surface/call.cc","file_line":1055,"grpc_message":"Exception calling application: local variable 'detection_threshold' referenced before assignment","grpc_status":2}"
>

In [ ]:
#View one output
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
if detection_options.detect_log is not None:
    from glob import glob
    idir = os.path.join(detection_options.results_root_path, detection_options.detect_log, 'image')
    bbox_imgs = glob(os.path.join(idir,'*'))
    for each_img_path in bbox_imgs:
        print(each_img_path)
        img = mpimg.imread(each_img_path)
        plt.imshow(img)
        plt.show()
        time.sleep(0.5)

In [ ]:
if detection_options.face_log is not None:
    from glob import glob
    bbox_imgs =  glob(os.path.join(detection_options.results_root_path, detection_options.face_log, 'image','*'))
    img_path = None
    for each_img_path in bbox_imgs:
        print(each_img_path)
        img = mpimg.imread(each_img_path)
        plt.imshow(img)
        plt.show()
        time.sleep(0.5)

In [ ]:
def process_video_detections(each):
    """
    Detections returned by the detector are written into a csvfile
    Frames annotated with bouding box are saved to a directory if options.detect_log is True
    Face region is extracted and resized to 128*128*3 and save to a directory of options.face_log is True
    Only frames with detections are saved
    """
    global video_header_flag
    im, results, options, file_identifier, media_type = each
    if results.done():
        recs = results.result().face_records
        i = 0
        detect_log_dir = None
        frame_id = None
        dimg = None
        for idx, face in enumerate(recs):
            base_name, ext = os.path.splitext(os.path.basename(face.source))
            # Filter faces based on min size
            size = min(face.detection.location.width,face.detection.location.height)
            if size < options.min_size:
                continue
    
            if video_header_flag:
                csv_header = ['source','frame','detect_id','type','score','x','y','w','h']
                if len(face.landmarks) > 0:
                    for each_lpt in face.landmarks:
                        pt_id_label = each_lpt.landmark_id
                        xpt_label = pt_id_label + '_x'
                        ypt_label = pt_id_label + '_y'
                        csv_header.append(pt_id_label)
                        csv_header.append(xpt_label)
                        csv_header.append(ypt_label)
                file_identifier.writerow(csv_header)
                video_header_flag = False
            
            csv_eachline = [face.source,
                            face.frame,
                            i,
                            face.detection.detection_class,
                            face.detection.score,
                            face.detection.location.x,
                            face.detection.location.y,
                            face.detection.location.width,
                            face.detection.location.height]

            if len(face.landmarks) > 0:
                for each_lpt in face.landmarks:
                    pt_id_label = each_lpt.landmark_id
                    xpt_label = each_lpt.location.x
                    ypt_label = each_lpt.location.y
                    csv_eachline.append(pt_id_label)
                    csv_eachline.append(xpt_label)
                    csv_eachline.append(ypt_label)
                    
            file_identifier.writerow(csv_eachline)
                
            if options.detect_log:
                detect_log_dir = os.path.join(options.results_root_path, options.detect_log, media_type, base_name)
                frame_id = face.frame
                if not os.path.exists(detect_log_dir):
                    os.makedirs(detect_log_dir, exist_ok=True)
                
                rect = pt.rect_proto2pv(face.detection.location)
                if dimg is None:
                    dimg = pv.Image(im[:,:,::-1])
                dimg.annotateThickRect(rect)
                dimg.annotateLabel(pv.Point(rect.x+5,rect.y+5),face.detection.detection_class)
                dimg.annotateLabel(pv.Point(rect.x+5,rect.y+20),
                                    "Score: %0.4f"%(face.detection.score,), color='yellow')
                if len(face.landmarks) > 0:
                    for each_lmark in face.landmarks:
                            dimg.annotateCircle(pv.Point(each_lmark.location.x, each_lmark.location.y), 
                                                radius=3, color = 'green', fill='green')
                
            
            if options.face_log:
                face_log_dir = os.path.join(options.results_root_path, options.face_log, media_type, base_name)
                if not os.path.exists(face_log_dir):
                    os.makedirs(face_log_dir, exist_ok=True)
                
                rect = pt.rect_proto2pv(face.detection.location)
                rect = rect.rescale(1.5)
                affine = pv.AffineFromRect(rect,(128,128))
                try:    
                    pvim = pv.Image(im[:,:,::-1])
                    view = affine(pvim)
                    out_path = os.path.join(face_log_dir,os.path.basename(base_name)+
                                            '_Frame_%06d'%(face.frame) +
                                            '_face_%03d'%(face.detection.detection_id,)+ '.jpg')
                    if len(face.landmarks) > 0:
                        for each_lmark in face.landmarks:
                            transformed_lmarks = affine(pv.Point(each_lmark.location.x, each_lmark.location.y))
                            view.annotateCircle(transformed_lmarks, radius=3, color = 'green', fill='green')
                    view.asAnnotated().save(out_path)
                    #print('Saving face:',out_path)
                except:
                    print("WARNING: Image not processed correctly:",face.source)
            i += 1
        
        if options.detect_log and detect_log_dir is not None:
            dimg.asAnnotated().save(os.path.join(detect_log_dir,
                                                 os.path.basename(base_name) + '_Frame_%06d'%(frame_id) + '.jpg'))
        return False
    return True



In [ ]:
def process_videos(each_video, doptions):
    """
    Looping through each frame in a video and making client calls to the service
    """
    detect_queue = []
    #Read Video
    video = pv.Video(each_video)
    videoname, ext = os.path.splitext(os.path.basename(each_video))
    fid = open(os.path.join(doptions.results_root_path,
                            doptions.csvfiles_log, 'video', videoname + '.csv') , 'w')
    video_detections_csv = csv.writer(fid)
    
    start_time = time.time()
    for frame_id, each_frame in enumerate(video):

        each_frame = each_frame.asOpenCV2()[:,:,::-1]#convert to opencv and then bgrtorgb
        if doptions.max_size is not None:
            each_frame = preprocessImage(each_frame, doptions)
        results = face_client.detect(each_frame, best=doptions.best, 
                             threshold=doptions.detect_thresh, 
                             min_size=doptions.min_size,
                             run_async=True, 
                             source=each_video, 
                             frame=frame_id + 1)
        detect_queue.append([each_frame, results, doptions, video_detections_csv, 'video'])
        detect_queue = list(filter(process_video_detections, detect_queue))

        
    while len(detect_queue):
        detect_queue = list(filter(process_video_detections,detect_queue))
        time.sleep(0.05)
    
    
    
    end_time = time.time()
    print(start_time, end_time, end_time - start_time)
    print("Processed %d frames in %0.3f seconds: %f images/second"%(frame_id+1,end_time - start_time,
                                                                    (frame_id+1)/(end_time - start_time)))
    
    fid.close()

In [ ]:
#loop through video list 
video_header_flag = None
if not os.path.isdir(os.path.join(detection_options.results_root_path, detection_options.csvfiles_log, 'video')):
    os.makedirs(os.path.join(detection_options.results_root_path, detection_options.csvfiles_log, 'video'))
for vid_num, each_video in enumerate(video_list):
    global video_header_flag 
    video_header_flag = True
    process_videos(each_video, detection_options)
    
    if detection_options.detect_log is not None:
        from glob import glob
        idir = os.path.join(detection_options.results_root_path, detection_options.detect_log, 
                            'video', os.path.splitext(os.path.basename(each_video))[0] )
        bbox_imgs = glob(os.path.join(idir,'*'))
        print('Number of Frames with Detection : ', len(bbox_imgs))
        for each_img_path in bbox_imgs:
            print(each_img_path)
            img = mpimg.imread(each_img_path)
            plt.imshow(img)
            plt.show()

In [ ]:
#read images from detect_log dir and convert it to a video and play it
